<a href="https://colab.research.google.com/github/casual-lab/MLC-Learn/blob/main/chapter2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
!python3 -m pip install mlc-ai-nightly -f https://mlc.ai/wheels

Looking in links: https://mlc.ai/wheels


第二章的主要内容是熟悉基本工具使用（TVM）。

总结来说，TVM提供了一个“算法”与“具体部署环境”之间的额外抽象层。使得算法和部署环境解耦。



In [18]:
import IPython
import numpy as np
import tvm
from tvm.ir.module import IRModule
from tvm.script import tir as T

In [19]:
# init data
a = np.arange(16).reshape(4, 4)
b = np.arange(16, 0, -1).reshape(4, 4)

In [20]:
# numpy version
c_np = a + b
c_np

array([[16, 16, 16, 16],
       [16, 16, 16, 16],
       [16, 16, 16, 16],
       [16, 16, 16, 16]])

In [21]:
# low-level numpy version
def lnumpy_add(a: np.ndarray, b: np.ndarray, c: np.ndarray):
  for i in range(4):
    for j in range(4):
      c[i, j] = a[i, j] + b[i, j]
c_lnumpy = np.empty((4, 4), dtype=np.int64)
lnumpy_add(a, b, c_lnumpy)
c_lnumpy

array([[16, 16, 16, 16],
       [16, 16, 16, 16],
       [16, 16, 16, 16],
       [16, 16, 16, 16]])

In [22]:
# TensorIR version
@tvm.script.ir_module
class MyAdd:
  @T.prim_func
  def add(A: T.Buffer((4, 4), "int64"),
          B: T.Buffer((4, 4), "int64"),
          C: T.Buffer((4, 4), "int64")):
    T.func_attr({"global_symbol": "add"})
    for i, j in T.grid(4, 4):
      with T.block("C"):
        vi = T.axis.spatial(4, i)
        vj = T.axis.spatial(4, j)
        C[vi, vj] = A[vi, vj] + B[vi, vj]

rt_lib = tvm.build(MyAdd, target="llvm")
a_tvm = tvm.nd.array(a)
b_tvm = tvm.nd.array(b)
c_tvm = tvm.nd.array(np.empty((4, 4), dtype=np.int64))
rt_lib["add"](a_tvm, b_tvm, c_tvm)
np.testing.assert_allclose(c_tvm.numpy(), c_np, rtol=1e-5)

# 2.5.1.2. 练习 1：广播加法¶


In [23]:
# init data
a = np.arange(16).reshape(4, 4)
b = np.arange(4, 0, -1).reshape(4)
a,b

(array([[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11],
        [12, 13, 14, 15]]),
 array([4, 3, 2, 1]))

In [24]:
# numpy version
c_np = a + b
c_np

array([[ 4,  4,  4,  4],
       [ 8,  8,  8,  8],
       [12, 12, 12, 12],
       [16, 16, 16, 16]])

In [25]:
@tvm.script.ir_module
class MyAdd:
  @T.prim_func
  def add(A: T.Buffer((4, 4), "int64"),
          B: T.Buffer((4,), "int64"),
          C: T.Buffer((4, 4), "int64")):
    T.func_attr({"global_symbol": "add"})
    for i, j in T.grid(4, 4):
      with T.block("C"):
        vi = T.axis.spatial(4, i)
        vj = T.axis.spatial(4, j)
        C[vi, vj] = A[vi, vj] + B[vj]

rt_lib = tvm.build(MyAdd, target="llvm")
a_tvm = tvm.nd.array(a)
b_tvm = tvm.nd.array(b)
c_tvm = tvm.nd.array(np.empty((4, 4), dtype=np.int64))
rt_lib["add"](a_tvm, b_tvm, c_tvm)
np.testing.assert_allclose(c_tvm.numpy(), c_np, rtol=1e-5)

# 2.5.1.3. 练习 2：二维卷积¶


In [26]:
N, CI, H, W, CO, K = 1, 1, 8, 8, 2, 3
OUT_H, OUT_W = H - K + 1, W - K + 1
data = np.arange(N*CI*H*W).reshape(N, CI, H, W)
weight = np.arange(CO*CI*K*K).reshape(CO, CI, K, K)

In [27]:
# torch version
import torch

data_torch = torch.Tensor(data)
weight_torch = torch.Tensor(weight)
conv_torch = torch.nn.functional.conv2d(data_torch, weight_torch)
conv_torch = conv_torch.numpy().astype(np.int64)
conv_torch

array([[[[ 474,  510,  546,  582,  618,  654],
         [ 762,  798,  834,  870,  906,  942],
         [1050, 1086, 1122, 1158, 1194, 1230],
         [1338, 1374, 1410, 1446, 1482, 1518],
         [1626, 1662, 1698, 1734, 1770, 1806],
         [1914, 1950, 1986, 2022, 2058, 2094]],

        [[1203, 1320, 1437, 1554, 1671, 1788],
         [2139, 2256, 2373, 2490, 2607, 2724],
         [3075, 3192, 3309, 3426, 3543, 3660],
         [4011, 4128, 4245, 4362, 4479, 4596],
         [4947, 5064, 5181, 5298, 5415, 5532],
         [5883, 6000, 6117, 6234, 6351, 6468]]]])

In [28]:
@tvm.script.ir_module
class MyConv:
  @T.prim_func
  def conv(D: T.buffer((N, CI, H, W), "int64"),
           W: T.buffer((CO, CI, K, K), "int64"),
           O: T.buffer((N, CO, OUT_H, OUT_W), "int64")):
    T.func_attr({"global_symbol": "conv", "tir.noalias": True})
    for n, co, h, w in T.grid(N, CO, OUT_H, OUT_W):
        with T.block("O"):
            vn = T.axis.spatial(N, n)
            vco = T.axis.spatial(CO, co)
            vh = T.axis.spatial(OUT_H, h)
            vw = T.axis.spatial(OUT_W, w)
            with T.init():
                O[vn, vco, vh, vw] = 0
            for ci, kh, kw in T.grid(CI, K, K):
                with T.block("C"):
                    vcci = T.axis.spatial(CI, ci)
                    vkh = T.axis.reduce(K, kh)
                    vkw = T.axis.reduce(K, kw)
                    O[vn, vco, vh, vw] = O[vn, vco, vh, vw] + \
                        D[vn, vcci, vh + vkh, vw + vkw] * W[vco, vcci, vkh, vkw]


rt_lib = tvm.build(MyConv, target="llvm")
data_tvm = tvm.nd.array(data)
weight_tvm = tvm.nd.array(weight)
conv_tvm = tvm.nd.array(np.empty((N, CO, OUT_H, OUT_W), dtype=np.int64))
rt_lib["conv"](data_tvm, weight_tvm, conv_tvm)
np.testing.assert_allclose(conv_tvm.numpy(), conv_torch, rtol=1e-5)

# 2.5.2.2. 练习 3：变换批量矩阵乘法程序¶

In [ ]:
def lnumpy_mm_relu_v2(A: np.ndarray, B: np.ndarray, C: np.ndarray):
    Y = np.empty((16, 128, 128), dtype="float32")
    for n in range(16):
        for i in range(128):
            for j in range(128):
                for k in range(128):
                    if k == 0:
                        Y[n, i, j] = 0
                    Y[n, i, j] = Y[n, i, j] + A[n, i, k] * B[n, k, j]
    for n in range(16):
        for i in range(128):
            for j in range(128):
                C[n, i, j] = max(Y[n, i, j], 0)

In [30]:
@tvm.script.ir_module
class MyBmmRelu:
  @T.prim_func
  def bmm_relu(A: T.buffer((16, 128, 128), "int64"),
               B: T.buffer((16, 128, 128), "int64"),
               C: T.buffer((16, 128, 128), "int64")):
    T.func_attr({"global_symbol": "bmm_relu", "tir.noalias": True})
    for n, i, j in T.grid(16, 128, 128):
        with T.block("C"):
            vn = T.axis.spatial(16, n)
            vi = T.axis.spatial(128, i)
            vj = T.axis.spatial(128, j)
            with T.init():
                C[vn, vi, vj] = 0
            for k in T.serial(0, 128):
                with T.block("C_1"):
                    vk = T.axis.reduce(128, k)
                    C[vn, vi, vj] = C[vn, vi, vj] + A[vn, vi, vk] * B[vn, vk, vj]
            with T.block("C_2"):
                C[vn, vi, vj] = T.max(C[vn, vi, vj], 0)

sch = tvm.tir.Schedule(MyBmmRelu)
IPython.display.Code(sch.mod.script(), language="python")
# Also please validate your result

# from tvm.script import ir as I
# from tvm.script import tir as T

@I.ir_module
class Module:
    @T.prim_func
    def bmm_relu(A: T.Buffer((16, 128, 128), "int64"), B: T.Buffer((16, 128, 128), "int64"), C: T.Buffer((16, 128, 128), "int64")):
        T.func_attr({"tir.noalias": T.bool(True)})
        # with T.block("root"):
        for n, i, j in T.grid(16, 128, 128):
            with T.block("C"):
                vn, vi, vj = T.axis.remap("SSS", [n, i, j])
                T.reads(A[vn, vi, 0:128], B[vn, 0:128, vj])
                T.writes(C[vn, vi, vj])
                with T.init():
                    C[vn, vi, vj] = T.int64(0)
                for k in range(128):
                    with T.block("C_1"):
                        vk = T.axis.reduce(128, k)
                        T.reads(C[vn, vi, vj], A[vn, vi, vk], B[vn, vk, vj])
                        T.writes(C[vn, vi, vj])
                        C[vn, vi, vj] = C[vn, vi, vj] + A[vn, vi, vk] * B[vn, vk, vj]
                with T.block("C_2"):
                    T.reads(C[vn, vi, vj])
                    T.writes(C[vn, vi, vj])
                    C[vn, vi, vj] = T.max(C[vn, vi, vj], T.int64(0))

In [ ]:
sch = tvm.tir.Schedule(MyBmmRelu)
# TODO: transformations
# Hints: you can use
# `IPython.display.Code(sch.mod.script(), language="python")`
# or `print(sch.mod.script())`
# to show the current program at any time during the transformation.

# Step 1. Get blocks
Y = sch.get_block("Y", func_name="bmm_relu")
...

# Step 2. Get loops
b, i, j, k = sch.get_loops(Y)
...

# Step 3. Organize the loops
k0, k1 = sch.split(k, ...)
sch.reorder(...)
sch.compute_at/reverse_compute_at(...)
...

# Step 4. decompose reduction
Y_init = sch.decompose_reduction(Y, ...)
...

# Step 5. vectorize / parallel / unroll
sch.vectorize(...)
sch.parallel(...)
sch.unroll(...)
...

IPython.display.Code(sch.mod.script(), language="python")